In [1]:
from functools import total_ordering
import streamlit as st
import pandas as pd
import numpy as np
from decimal import *
import openpyxl

In [2]:
state_name = pd.read_csv("states.csv")
el_price = pd.read_excel("el_price_rev-v2.xlsx")
el_price_n = pd.read_excel("el_price_rev_add.xlsx") #with microgrid and solar rooftop
usd_inr = 82.23 # 1 usd = 82.23 INR

In [5]:
df_base_cf = pd.read_excel("cook-fuel-stove.xlsx",sheet_name="baseline") ##area_select
df_base_cf2 = df_base_cf.loc[(df_base_cf['Area'] == 'Rural') & (df_base_cf['Socio-Economic'] == 'Middle')]
df_base_cf1 = df_base_cf2["Fuel"].drop_duplicates()
bpf_select = 'LPG'
mask_bf_stv = df_base_cf['Fuel'].isin([bpf_select])
df_base_cf_st = df_base_cf2[mask_bf_stv].iloc[:,2]

/var/folders/my/_04ctykd2038d4bbkcv7bk980000gn/T/ipykernel_48641/48514849.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_base_cf_st = df_base_cf2[mask_bf_stv].iloc[:,2]


In [6]:
df_elec_cf = pd.read_excel("cook-fuel-stove.xlsx",sheet_name="e-cooking")
df_elec_cf2 = df_elec_cf.loc[(df_elec_cf['Area'] == 'Rural') & (df_elec_cf['Socio-Economic'] == 'Middle')]
df_elec_cf1 = df_elec_cf2["Fuel"].drop_duplicates()
epf_select = 'Grid electricity'
mask_ef_stv = df_elec_cf['Fuel'].isin([epf_select])
df_elec_cf_st = df_elec_cf2[mask_ef_stv].iloc[:,2]

/var/folders/my/_04ctykd2038d4bbkcv7bk980000gn/T/ipykernel_48641/805956605.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_elec_cf_st = df_elec_cf2[mask_ef_stv].iloc[:,2]


In [ ]:
encons_var  = {'Variable':['Daily cooking duration', 'Hourly consumption', 'Daily consumption', 'Annual consumption','Unit carbon emission','Annual carbon emission'],
	'Units':['hours/day','kWh/hour','kWh/day','kWh/year','MtCO2eq./kWh','MtCO2eq./year'],
	'Baseline':	[bs_dcd.round(2),bs_hc.round(2),bs_dc.round(2),(bs_dc * 365 * 0.9).round(2),bs_uce.round(2),bs_tce.round(2)],
	'e-Cooking':[el_dcd.round(2),el_hc,el_dc.round(2),el_ac.round(2),el_uce.round(2),el_tce.round(2)],
	'Delta':[(el_dcd - bs_dcd).round(2),(el_hc - bs_hc).round(2),(el_dc - bs_dc).round(2),(el_ac - bs_ac).round(2),(el_uce- bs_uce).round(2),(el_tce - bs_tce).round(2)]
	}
	df_encons_var = pd.DataFrame(encons_var)
	df_encons_var_round = df_encons_var.round(2)
	#st.dataframe(df_encons_var)

# with st.expander('6\) Results  \- Energy Cost'):
	#baseline - energy cost
	bs_uc = df_base_cf4['Unit cost'].values[0] #unit cost
	bs_uc = np.array(bs_uc)
	#bs_opc = df_base_cf4['Opex'].values[0] #opex
	bs_opc = bs_uc * bs_ac
	bs_ovc = df_base_cf4['Overheads (Fuel)'].values[0] #overheads
	bs_etc = bs_opc + bs_ovc #total cost
	bs_scc = (86 * usd_inr * bs_tce).round(0) #social cost of carbon

	#electric cost
	el_price2 = el_price[['State',soc_eco_select]] #filtering dataframe
	#el_price3 = el_price2.loc[el_price2['State'] == state_select, soc_eco_select] #unit cost

	### revision to include microgrid and solar rooftop
	#el_price3_1 = el_price2.loc[el_price2['State'] == state_select,el_price2['Source'] == epf_select,soc_eco_select] #unit cost
	if epf_select == 'Microgrid':
		el_price3 = 20
		el_price3_1 = 20
	elif epf_select == "Solar rooftop":
		el_price3 = 0
		el_price3_1 = 0
	else:
		el_price3 = el_price2.loc[el_price2['State'] == state_select, soc_eco_select].values #unit cost
		#el_price3 = el_price3.astype(np.float64)
		el_price3_1 = str(el_price3).replace(' [', '').replace('[', '').replace(']', '')
		#el_price3_list = el_price3.flatten().tolist()
		#el_price3 = el_price2.loc[el_price2['State'] == state_select, soc_eco_select].to_string #unit cost

	#st.markdown(type(el_price3))
	#st.markdown(type(el_ac))
	#st.markdown(str(el_price3).replace(' [', '').replace('[', '').replace(']', ''))
	#el_price_n1 = el_price_n['State',soc_eco_select] #filtering dataframe
	#el_uc_n2 = el_price4.item()
	el_uc = el_price3
	el_uc_1 = el_price3_1
	#el_uc = df_el_cf4['Unit cost'].values[0]
	#el_opc = df_el_cf4['Opex'].values[0] #opex
	#el_ac = np.array(el_ac)
	el_opc = el_uc * el_ac
	el_opc_1 = str(el_opc.round(2)).replace(' [', '').replace('[', '').replace(']', '')
	el_ovc = df_el_cf4['Overheads (Fuel)'].values[0] #overheads
	el_etc = el_opc + el_ovc #total operating cost
	el_etc_1 = str(el_etc.round(2)).replace(' [', '').replace('[', '').replace(']', '')
	el_scc = (86 * usd_inr * el_tce).round(0) #social cost of carbon

	#delta calc
	d_uc = (el_uc - bs_uc).round(2)
	d_uc1 = str(d_uc).replace(' [', '').replace('[', '').replace(']', '')

	d_opc = (el_opc - bs_opc).round(2)
	d_opc1 = str(d_opc).replace(' [', '').replace('[', '').replace(']', '')

	d_etc = (el_etc - bs_etc).round(2)
	d_etc1 = str(d_etc).replace(' [', '').replace('[', '').replace(']', '')

	#dataframe - energy cost
	encost_var  = {'Variable':['Unit cost', 'Opex', 'Overheads', 'Total operating cost','Social Carbon Cost'],
	'Units':['INR/kWh','INR/year','INR/year','INR/year','INR/year'],
	'Baseline':[bs_uc,bs_opc.round(2),bs_ovc,bs_etc.round(2),bs_scc],
	'e-Cooking':[el_uc_1,el_opc_1,el_ovc,el_etc_1,el_scc],
	'Delta':[d_uc1,d_opc1,(el_ovc - bs_ovc).round(2),d_etc1, (el_scc - bs_scc)]
	}
	df_encost_var = pd.DataFrame(encost_var)
	df_tcost = pd.concat([df_encons_var,df_encost_var], ignore_index=True)
	df_tcost = df_tcost.astype(str)
	#df_tcost = df_tcost.applymap(lambda x: "{:.2f}".format(x))
	#st.write(df_tcost)
	
	sty_df_tcost = df_tcost.style.apply(highlight_negative, axis = 1)
	# st.dataframe(df_tcost)
	st.dataframe(sty_df_tcost)


In [11]:
import pandas as pd

# create a sample DataFrame
df = pd.DataFrame({'A': [1, -2, 3, -4], 'B': [5, 6, -7, 8], 'C': [-9, 10, 11, 12]})

# define a function to highlight negative values
def highlight_negative(val):
    color = 'background-color: yellow' if val < 0 else ''
    return color

# apply the function to rows where 'A' < 0 or 'B' < 0
highlighted_df = df.style.apply(lambda x: [highlight_negative(x.loc[row]) if (x.loc[row, 'A'] < 0 or x.loc[row, 'B'] < 0) else ['']*len(x.columns) for row in x.index], axis=1)

# display the highlighted DataFrame
highlighted_df


IndexingError: Too many indexers